<div style="text-align:center; font-size:30px">llama 3 Vs Chat GPT 3.5
</br>
<img src="./llama3_Vs_GPT_3_5.png">
</div>

- # 1. Introduction :
    - ### 1.1 Objective : 
        - This project aims and compare the responses from Two diffrent large language models : llama3 and Chat GPT 3.5 turbo
    - ### 1.2 Specs :
        -  llama3 : installed on local machine the latest 8b model @ 06/03/2024
        -  Chat gpt : using a private API key, the model is gpt-3.5-turbo
    - ### 1.3 Installation :
        -  llama3 :
            -  Link to Ollama tool : <a href="https://ollama.com/download">Ollama</a>
            -  Link to the llm model : <a href="https://ollama.com/library/llama3">llama3</a>
            -  After installing the ollama tool : download model by the following command
                -  ollama run llama3
        -  Gpt :
            -  How to use gpt : you need to have a subscription to use their API
            -  Call the OPENAI api : you need to add a payment method to avoid api limit reached error code 429
                -  Click here <a href="https://platform.openai.com/playground" >API Dashboard</a>
                -  After the payment is added, go to usage section on left panel and see if the bar is green, which means you can call the api
- # 2. Test Scenarios :
    -  The <a href="https://github.com/SouLayman2022/llama3_Vs_Gpt_3.5">repos</a> contains a page saved as a pdf, link to real website <a href="https://www.caranddriver.com/tesla/model-3">link</A>              
    -  We wwill be using both models to see which one of them gets the response right, in regards of the prompts at the end of this notebook
    - The MODEL variable holds the llm models names, so comment llama3 if you want to use gpt, else leave it as is the run llama3 instead
- # 3. Running the notebook :
    - open the file PDF_RAG.ipynb on your IDE, Vscode... or use the command on cmd like this :
        - git clone https://github.com/SouLayman2022/llama3_Vs_Gpt_3.5.git
        - cd llama3_Vs_Gpt_3.5
        - python -m venv llama3_requirements
        - llama3_requirements\Scripts\ativate
        - pip install -r requirements.txt
        - jupyter notebook
        - a web page contaning the noteboo will be shwon double click the : PDF_RAG.ipynb to open it
        - after finishing running or twiking do following commands :
            - pip freeze > requirements.txt ( save your installed packages )
            - deactivate ( to opt out of venv )

In [1]:
import os
from dotenv import load_dotenv
from tqdm import tqdm # know my code progress
#openai api
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
#llama3 locally
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# comment out the MODEL="llama3", if you want to use llama3 else leave it as a comment and use GPT3.5
MODEL = "gpt-3.5-turbo"
MODEL = "llama3"


if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY, model = MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("tell me a joke")

"Here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nHope that made you smile!"

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke ("tell me a joke") 

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!\n\nHope that made you laugh! Do you want to hear another one?"

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Tesla_3.pdf")
pages = loader.load_and_split()
pages

[Document(page_content="Home / Tesla / M Mo od de el l  3 3\n2024 Tesla Model 3\nStarting at $40,630S SP PE EC CS SP PH HO OT TO OS S\nTESLA20248 8/10\nC/D RATINGOVERVIEWRANKWHAT'S NEWPRICINGEV MOTOR & PERFORMANCE0-60 TIMESUBSCRIBEREVIEWSNEWSWHAT'S MY CAR WORTH?BUYER'S GUIDE2024 Tesla Model 3 Review, Pricing, and Specs https://www.caranddriver.com/tesla/model-3\n1 of 13 06/03/2024, 11:36", metadata={'source': 'Tesla_3.pdf', 'page': 0}),
 Document(page_content="H HI IG GH HS S\nEntertaining handling, speedy acceleration, one of the best small EVs for driving range.\nL LO OW WS S\nIrritating lack of physical controls for key functions, advertised range is diﬃcult to achieve, even\nafter all these years the AutoPilot system still needs polish.\nV VE ER RD DI IC CT T\nIt's aﬀordable, desirable, and fun to drive, which makes the Model 3 an attractive prospect for those\nswitching to EV motoring.\nBY DREW DORIAN\nO Ov ve er rv vi ie ew w\nWith a sleek new look, an attractive estimated drivin

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. if you can't
Answer the question. reply "Sorry i don't know the answer, try contacting admin / soulayman or looking on web for answers"

Context: {context}

Question:{question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. if you can't
Answer the question. reply "Sorry i don't know the answer, try contacting admin / soulayman or looking on web for answers"

Context: Here is some context

Question:Here is a question



In [5]:
chain = prompt | model | parser

In [6]:
#chain schema explained
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [7]:
chain.invoke(
    {
        "context": "My name is Soulayman, i'm the admin",
        "question": "Who am i, and what i do?"
    }
)

"I've got this one!\n\nYou are Soulayman, and you're the admin!"

In [8]:
chain.invoke(
    {
        "context": "My age is 27 after 10 years how old ma i",
        "question": "answer the question"
    }
)

"I'd be happy to help!\n\nSince your current age is 27 and you're asking what your age will be after 10 years, that means we need to add 10 to your current age.\n\n27 + 10 = 37\n\nSo, after 10 years, you'll be 37 years old!"

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
        pages,
        embedding = embeddings
    )

In [10]:
retriever = vectorstore.as_retriever()

retriever.invoke("Tesla model 3")

[Document(page_content="I In nt te er ri io or r, ,  C Co om mf fo or rt t, ,  a an nd d  C Ca ar rg go o\nWe're not exaggerating when we say that the Tesla Model 3 has an interior unlikeany other car  on the market today. It's shockingly simple inside, with nearly\neverything controlled by the monolithic touchscreen in the center of the dashboard.\nButtons on the steering wheel control things like the exterior side view mirrors, the\nposition of the steering column, and even the turn signals—and we wish those\ncontrols were of the conventional design instead. The Tesla's low, \uf002at \uf002oor makes for\na spacious and airy feel inside. The front seats are supportive and comfortable, but\nthe rear seats are positioned low and are cramped and uncomfortable; don't expect\nadults to be able to spend long periods of time in them. Folding the 60/40 split-\nfolding rear seats is simple and expands the trunk space considerably. The Model 3's\nseatbacks fold \uf002at, too, providing an unint

In [11]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
     }
    | prompt
    | model
    | parser
)

chain.invoke({"question" : "What is Tesla model 3?"})

'Based on the provided context, it appears that Tesla Model 3 is an electric vehicle with a starting price of $40,630. It has various trim levels and options, including the Performance trim which boasts 510 HP and a 0-60 mph time of just 2.9 seconds. The article also discusses its features, range, and pricing in comparison to other electric vehicles on the market.'

In [12]:
# Put your prompts / questions here

questions = [
"What topic the document is about ?",
"What is the name of the website in the document ?",
"What is name of the car ?",
"What is the price of the car ?",
"What is the year of production of the car ?",
"How much HP does the car have ?",
"How much time it takes the car to reach 60 mph ?",
"How much is the range of the car ?",
"How much is the the fuel rating of the car ?",
]

for question in questions:
    print(f"Question : {question}")
    print(f"Answer: {chain.invoke({"question" : question})}\n")
    print("#"*30)

Question : What topic the document is about ?
Answer: Based on the context, I can confidently say that the topic of the document is about the Tesla Model 3, specifically the 2024 model, its features, specifications, and review. The document appears to be a car review or specification sheet for the Tesla Model 3.

##############################
Question : What is the name of the website in the document ?
Answer: According to the context, the website mentioned in the document is:

https://www.caranddriver.com/

This is mentioned multiple times throughout the documents.

##############################
Question : What is name of the car ?
Answer: The name of the car is Tesla Model 3.

##############################
Question : What is the price of the car ?
Answer: According to the context, the prices of the different models are as follows:

* Tesla Model 3 (base) - starting at $40,630
* Tesla Model 3 Performance - starting at $51,300
* Polestar 2 - starting at $51,100
* BMW i4 - starting a